In [2]:
import pandas as pd
import numpy as np
import censusgeocode as cg
import time
from datetime import datetime

import geopandas as gpd
import os

### Data loading functions

1. Clinic/Center - Amputee: 261QA0900X
2. Orthotist: 222Z00000X
3. Prosthetist: 224P00000X
4. Prosthetic/Orthotic Supplier:335E00000X

In [56]:
states = ("AL","AK","AZ","AR","CA","CO","CT","DE","DC","FL","GA","HI","ID",
          "IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO",
          "MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA",
          "RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY")

keep_col = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)',
            'NPI Deactivation Reason Code','NPI Deactivation Date','NPI Reactivation Date',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']

taxon_codes = ['Healthcare Provider Taxonomy Code_' + str(i+1) for i in range(15)]
taxonswitch_codes = ['Healthcare Provider Primary Taxonomy Switch_' + str(i+1) for i in range(15)]
keep_col += taxon_codes
keep_col += taxonswitch_codes
community_pharm = ['261QA0900X','222Z00000X', '224P00000X','335E00000X']
npi_csv = 'npidata_pfile_20050523-20230212.csv'

In [57]:
def sub_rows(data, state):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == state
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta & ac 
    sub = data[all_together]
    return sub

def csv_chunks(file,chunk_size,keep_cols,state,row_sub):
    header_fields = list(pd.read_csv(npi_csv, nrows=1))
    header_locs = [header_fields.index(i) for i in keep_cols]
    skip = 1
    it_n = 0
    sub_n = 0
    ret_chunk = chunk_size
    fin_li_dat = []
    while ret_chunk == chunk_size:
        file_chunk = pd.read_csv(file, usecols=header_locs, skiprows=skip, 
                     nrows=chunk_size, names=header_fields, dtype='str')
        sub_dat = row_sub(file_chunk, state)
        fin_li_dat.append( sub_dat.copy() )
        skip += chunk_size
        it_n += 1
        sub_n += sub_dat.shape[0]
        print(f'Grabbed iter {it_n} total sub n so far {sub_n}')
        ret_chunk = file_chunk.shape[0]
    fin_dat = pd.concat(fin_li_dat, axis=0)
    return fin_dat

### Combine

In [63]:
ph_txs = pd.DataFrame()

In [81]:
dfs = []

for s in states:
    print(s)
    print(datetime.now())
    pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, state = s, row_sub=sub_rows)
    print(datetime.now()) 
    dfs.append(pharm_tx)

AL
2023-05-01 04:41:56.244558
Grabbed iter 1 total sub n so far 6
Grabbed iter 2 total sub n so far 36
Grabbed iter 3 total sub n so far 65
Grabbed iter 4 total sub n so far 86
Grabbed iter 5 total sub n so far 97
Grabbed iter 6 total sub n so far 125
Grabbed iter 7 total sub n so far 152
Grabbed iter 8 total sub n so far 163
2023-05-01 04:43:58.082195
AK
2023-05-01 04:43:58.082253
Grabbed iter 1 total sub n so far 2
Grabbed iter 2 total sub n so far 6
Grabbed iter 3 total sub n so far 11
Grabbed iter 4 total sub n so far 11
Grabbed iter 5 total sub n so far 11
Grabbed iter 6 total sub n so far 13
Grabbed iter 7 total sub n so far 17
Grabbed iter 8 total sub n so far 17
2023-05-01 04:46:02.169932
AZ
2023-05-01 04:46:02.169959
Grabbed iter 1 total sub n so far 7
Grabbed iter 2 total sub n so far 35
Grabbed iter 3 total sub n so far 52
Grabbed iter 4 total sub n so far 77
Grabbed iter 5 total sub n so far 101
Grabbed iter 6 total sub n so far 142
Grabbed iter 7 total sub n so far 178
Gra

In [82]:
combined = pd.concat(dfs)
len(combined)

In [ ]:
end_str = [' STE', ' SUITE', ' BLDG', ' TOWER', ', #', ' UNIT',
           ' APT', ' BUILDING',',', '#']

def clean_add(address):
    add_new = address.upper()
    for su in end_str:
        sf = address.find(su)
        if sf > -1:
            add_new = add_new[0:sf]
    add_new = add_new.replace('.','')
    add_new = add_new.strip()
    return add_new

In [9]:
combined['Provider Business Practice Location Address Postal Code'] = combined['Provider Business Practice Location Address Postal Code'].str[0:5]
combined['Zip5'] = combined['Provider Business Practice Location Address Postal Code'].str[0:5]
combined['Address'] = combined['Provider First Line Business Practice Location Address'].apply(clean_add)

combined.rename(columns={'Provider Business Practice Location Address City Name':'City',
                      'Provider Business Practice Location Address State Name':'State2'},
             inplace=True)
combined = combined[-combined['Provider Organization Name (Legal Business Name)'].str.contains("NORDSTROM")].reset_index(drop = True)
combined.head(2)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,City,State2,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number,Zip5,Address,taxonomy
0,1447443551,2,"ALABAMA PROSTHETICS & ORTHOTICS, INC",101 MEDICAL CENTER CT,PRATTVILLE,AL,36066,3.343620e+09,36066,101 MEDICAL CENTER CT,335E00000X
1,1831369404,2,"ORTHOPRO, INC",1812 28TH AVE S,BIRMINGHAM,AL,35209,2.058799e+09,35209,1812 28TH AVE S,335E00000X


In [10]:
newc = []
for n in range(len(combined)):
    for i in range(15):
        if combined.iloc[n]['Healthcare Provider Primary Taxonomy Switch_' + str(i+1)] == 'Y':
            v = combined.iloc[n]['Healthcare Provider Taxonomy Code_' + str(i+1)]
    newc.append(v)

KeyError: 'Healthcare Provider Primary Taxonomy Switch_1'

In [86]:
dec_codes = ['NPI Deactivation Reason Code','NPI Deactivation Date','NPI Reactivation Date']
combined = combined.drop(columns=taxon_codes+taxonswitch_codes+dec_codes).reset_index(drop=True)
combined['taxonomy'] = newcs
# combined.to_csv('combinedDf.csv',index=False)

### Load

In [3]:
combined = pd.read_csv('combinedDf.csv')

In [4]:
combined2 = combined[combined['taxonomy'].str.contains("261QA0900X")]

In [5]:
combined = combined[combined['Provider Organization Name (Legal Business Name)'].str.contains("ORTHO|PROS|P&O|LIMB")].reset_index(drop = True)
len(combined)

4375

In [8]:
combined_3 = pd.concat([combined, combined2]).drop_duplicates()
combined_3.shape

(4375, 11)

In [9]:
combined_3 = combined_3.sort_values(by=['State2'])
len(combined_3["Address"].unique())

4062

In [93]:
combined_4 = combined_3[~combined_3.duplicated(subset=['Address'], keep="last")].reset_index(drop = True)

In [12]:
# combined_4.to_excel('us_clinics.xlsx',index=False)

In [22]:
# combined_4[combined_4['Provider Organization Name (Legal Business Name)'].str.contains("ABI")]

In [66]:
combined_5 = pd.DataFrame([combined_4.index, combined_4.Address, combined_4.City, combined_4.State2, combined_4.Zip5]).transpose()
combined_5.columns = ['index', 'Address', 'City', 'State2', 'Zip5']
combined_5

,index,Address,City,State2,Zip5
0,0,1405 KELLUM ST,FAIRBANKS,AK,99701
1,1,3719 E MERIDIAN LOOP,WASILLA,AK,99654
2,2,3400 LATOUCHE ST,ANCHORAGE,AK,99508
3,3,44604,SOLDOTNA,AK,99669
4,4,5701 LAKE OTIS PKWY,ANCHORAGE,AK,99507
...,...,...,...,...,...
4057,4057,135 S WASHINGTON ST,CASPER,WY,82601
4058,4058,611 E CARLSON STREET,CHEYENNE,WY,82009
4059,4059,240 W 9TH ST,CASPER,WY,82601
4060,4060,720 LINDSAY LN,CODY,WY,82414


In [67]:
combined_5.to_csv('cgDf.csv',index=False, header=False)

In [68]:
k = cg.addressbatch('cgDf.csv')

In [69]:
geo_pharm = pd.DataFrame(k, columns=k[0].keys())

In [70]:
print(geo_pharm['match'].value_counts())

True     3559
False     503
Name: match, dtype: int64


In [98]:
# geo_pharm['rowN'] = geo_pharm['row'].astype(int)
# gp2 = geo_pharm.sort_values(by='rowN').reset_index(drop=True)

kg = ['id','address','match','lat','lon']
kd = ['id','NPI',
      'Provider Organization Name (Legal Business Name)',
      'Provider Business Practice Location Address Telephone Number',
      'City','State2','Zip5']

In [99]:
geo_pharm['id'] = geo_pharm['id'].astype('int')

In [100]:
geo_pharm.sort_values('id', inplace=True)
# geo_pharm

In [101]:
combined_4['id'] = combined_4.index
# combined_4

In [106]:
final_pharm = pd.merge(combined_4[kd], geo_pharm[kg], on="id")
final_pharm.rename(columns={'Provider Organization Name (Legal Business Name)':'Name',
                      'Provider Business Practice Location Address Telephone Number':'Telephone'}, inplace=True)
# final_pharm.head(2)

In [ ]:
hosp_data = final_pharm.drop(columns = 'id')

In [113]:
hosp_data = hosp_data[hosp_data['match']].copy()
hosp_data.reset_index(inplace=True, drop=True)
hosp_data.head(2)

,NPI,Name,Telephone,City,State2,Zip5,address,match,lat,lon
0,1689920142,ALCHEMY ORTHOTICS AND PROSTHETICS,9.075621e+09,FAIRBANKS,AK,99701,"1405 KELLUM ST, FAIRBANKS, AK, 99701",True,64.837960,-147.747143
1,1952445041,NORTHERN ORTHOPEDICS INC,9.073574e+09,WASILLA,AK,99654,"3719 E MERIDIAN LOOP, WASILLA, AK, 99654",True,61.588159,-149.357567


In [115]:
# hosp_data[hosp_data['Name'].str.contains("ABI", na=False)]

In [116]:
hosp_geo = gpd.GeoDataFrame(hosp_data, geometry=gpd.points_from_xy(hosp_data.lon, hosp_data.lat), crs="EPSG:4326")

In [117]:
cali_counties = gpd.read_file(r'tl_2019_us_county/tl_2019_us_county.shp')
cali_outline = cali_counties.dissolve('STATEFP')
cali_proj = cali_outline.to_crs('EPSG:5070')
print(cali_outline.crs)

EPSG:4269


In [118]:
def dissolve_buff(point_df,d,resolution):
    bu = point_df.buffer(d,resolution)
    geodf = gpd.GeoDataFrame(geometry=bu)
    geodf['Const'] = 0
    single = geodf.dissolve('Const')
    return single[['geometry']]

In [119]:
def dist_cont(point_df,dist_list,outside,buff_res):
    if point_df.crs != outside.crs:
        print('Point df and Outside df are not the same CRS')
        return None
    # Making outside area out dissolved object
    out_cop = outside[['geometry']].copy()
    out_cop['Constant'] = 1
    out_cop = out_cop.dissolve('Constant')
    # Make sure points are inside area
    inside = point_df.within(out_cop['geometry'][1])
    point_cop = point_df[inside].copy()
    point_cop = point_df.copy()
    point_cop['Constant'] = 1 #Constant for dissolve
    point_cop = point_cop[['Constant','geometry']].copy()
    res_buffers = []
    for i,d in enumerate(dist_list):
        print(f'Doing buffer {d}')
        if i == 0:
            res = dissolve_buff(point_cop, d, buff_res)
            res_buffers.append(res.copy())
        else:
            res_new = dissolve_buff(point_cop, d, buff_res)
            res_buffonly = gpd.overlay(res_new, res, how='difference')
            res = res_new.copy()
            res_buffers.append( res_buffonly.copy() )
    # Now take the difference with the larger area
    print('Working on leftover difference now')
    leftover = gpd.overlay(out_cop, res, how='difference')
    res_buffers.append(leftover)
    for i,d in enumerate(dist_list):
        res_buffers[i]['Distance'] = str(d)
    res_buffers[-1]['Distance'] = 'Outside'
    # New geopandas DF
    comb_df = pd.concat(res_buffers)
    comb_df.reset_index(inplace=True, drop=True)
    return comb_df

In [120]:
hos_proj = hosp_geo.to_crs('EPSG:5070') #'epsg:4269'

dist_met = [2000, 4000, 8000, 16000] #, 32000
buff_city = dist_cont(hos_proj, dist_met, cali_proj, buff_res=100)

Doing buffer 2000
Doing buffer 4000
Doing buffer 8000
Doing buffer 16000
Working on leftover difference now


In [121]:
#Now making folium plot
buff_map = buff_city.to_crs('EPSG:4326')
kv = list(hosp_geo)[1:10]

In [122]:
#"fill": "#00aa22",
#"fill-opacity": 0.5

cols = ['#f1eef6',
'#d7b5d8',
'#df65b0',
'#dd1c77',
'#980043']

buff_map['fill'] = cols
buff_map['fill-opacity'] = 0.35

#os.chdir(r'D:\Dropbox\Dropbox\PublicCode_Git\Blog_Code')

In [123]:
buff_map.to_file('Combined_buff.geojson', driver='GeoJSON')
hosp_geo.to_file('Combined_States.geojson', driver='GeoJSON')